<a href="https://colab.research.google.com/github/Soharwardi/PIAIC-Agentic-AI/blob/main/real_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain pinecone-client sentence-transformers openai

In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document


In [ ]:
# Initialize Pinecone
pinecone.init(api_key="YOUR_PINECONE_API_KEY", environment = "us-west1-gcp")


In [ ]:
index_name = 'langchain-index'

# Check if the index already exists
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=1536,  # Adjust dimension as needed
        metric='cosine'
    )

# Connect to the index
index = pinecone.Index(index_name)


In [ ]:
# Load a Single Document
def load_document(file_path):
    docs = []
    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            text = file.read()
            docs.append(Document(text=text))
    return docs

documents = load_document("/content/1.txt")





In [ ]:
# Split Documents into Chunks
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = splitter.split_documents(documents)


In [ ]:
# Generate Embeddings
openai.api_key = "YOUR_OPENAI_API_KEY"
embeddings = OpenAIEmbeddings()
embeddings_docs = [embeddings.embed(doc.page_content) for doc in docs]

In [ ]:
# Store Embeddings in Pinecone
pinecone_vectorstore = LangchainPinecone(index=index, embeddings=embeddings_docs)
pinecone_vectorstore.add_documents(docs)


In [ ]:
# Set Up RetrievalQA Chain
qa = RetrievalQA(vectorstore=pinecone_vectorstore, retriever='semantic')

In [ ]:
# Ask Questions
question = " what is janat"
answer = qa.ask(question)
print(answer)
